In [1]:
import json
import os

In [2]:
data = []
with open('prepared-lom.agc.gov.my-llama3.jsonl') as fopen:
    for l in fopen:
        l = json.loads(l)
        if len(l['text']) < 512:
            continue
        data.append(l['text'])

In [3]:
len(data)

2608

In [4]:
folder = 'generate-lom.agc.gov.my-open-qa-llama3'

In [8]:
# !rm -rf generate-lom.agc.gov.my-open-qa-llama3
!mkdir {folder}

mkdir: cannot create directory ‘generate-lom.agc.gov.my-open-qa-llama3’: File exists


In [6]:
results = []
for i in range(len(data)):
    
    s = """
```
{{replace_me}}
```

Generate list of abstractive QA in malay, return in JSON [{'question', 'answer'}]
    """.strip()
    s = s.replace('{{replace_me}}', data[i])
    results.append(s)
        
len(results)

2608

In [9]:
from openai import OpenAI
import requests

headers = {
    'accept': 'application/json',
    'Content-Type': 'application/json',
}

def answer(q, i):
    filename = os.path.join(folder, f'{i}.json')
    if os.path.exists(filename):
        return
    
    results = []
    openai = OpenAI(
        base_url='https://llama-3.us.mesolitica.com/v1',
        api_key='empty',
    )
    for _ in range(1):
        
        while True:
            try:
                
                json_data = {
                    'messages': [
                        {
                            'role': 'user',
                            'content': q,
                        },
                    ],
                    'model': 'model',
                    'stop': [
                        '<|eot_id|>',
                    ],
                    'temperature': 0.9,
                    'max_tokens': 2048,
                }
                response = requests.post(
                    'https://llama-3.us.mesolitica.com/v1/chat/completions', 
                    headers=headers, json=json_data, timeout = 60 * 10)
                
                r = response.json()['choices'][0]['message']['content']
                results.append(r)
                break
            except Exception as e:
                # print(e)
                pass
                
    
    with open(filename, 'w') as fopen:
        json.dump(results, fopen)

In [10]:
def consumer(queue, name):
    while True:
        if queue.qsize() == 0:
            break
        item = queue.get()
        answer(*item)
    print(f'consumer {name} done')

In [11]:
urls = [(q, no) for no, q in enumerate(results)]

In [13]:
answer(*urls[0])

In [14]:
from threading import Thread
from queue import Queue
from tqdm import tqdm

queue = Queue()
for u in urls:
    queue.put(u)
    
ori_size = queue.qsize()

In [15]:
max_worker = 50
consumers = [Thread(target=consumer, args=(queue,i)) for i in range(max_worker)]
for i in range(len(consumers)):
    consumers[i].start()
    
pbar = tqdm(total=ori_size)
last_size = 0
while True:
    size = queue.qsize()
    if size == 0:
        break
    left = ori_size - size
    minus = left - last_size
    if minus > 0:
        pbar.update(minus)
        last_size += minus

pbar.close()

100%|██████████████████████████████████████████████████████████████████████████▉| 2607/2608 [1:21:44<00:01,  1.88s/it]


consumer 47 done
consumer 6 done
consumer 10 done
consumer 46 done
consumer 48 done
consumer 26 done
consumer 11 done
consumer 33 done
consumer 1 done
consumer 42 done
consumer 23 done
consumer 13 done
consumer 9 done
consumer 5 done
consumer 34 done
consumer 12 done
consumer 39 done
consumer 15 done
consumer 38 done
consumer 14 done
consumer 21 done
consumer 31 done
consumer 43 done
consumer 27 done
consumer 29 done
consumer 41 done
consumer 30 done
consumer 3 done
consumer 28 done
consumer 24 done
consumer 49 done
consumer 20 done
consumer 36 done
consumer 19 done
consumer 37 done
consumer 17 done
consumer 45 done
consumer 8 done
consumer 40 done
consumer 35 done
consumer 18 done
consumer 16 done
consumer 44 done
consumer 4 done
consumer 0 done
consumer 22 done
consumer 7 done
consumer 32 done
consumer 25 done
consumer 2 done


In [5]:
data = []
with open('prepared-lom.agc.gov.my-llama3.jsonl') as fopen:
    for l in fopen:
        l = json.loads(l)
        if len(l['text']) < 512:
            continue
        data.append(l)

In [6]:
!pip3.8 freeze | grep ctranslate

ctranslate2==3.23.0


In [7]:
suppose = {'question', 'answer'}

In [8]:
from tqdm import tqdm

with open('generate-lom.agc.gov.my-open-qa-llama3.jsonl', 'w') as fopen_j:
    for i in tqdm(range(len(data))):
        f = f'{folder}/{i}.json'
        if not os.path.exists(f):
            continue
            
        with open(f) as fopen:
            d = json.load(fopen)
        
        questions = []
        unique_questions = set()
        for d_ in d:
            try:
                selected = []
                for d__ in d_.split('```'):
                    try:
                        try:
                            selected.append(json.loads(d__))
                        except:
                            selected.append(eval(d__))
                    except:
                        pass
                if not len(selected):
                    continue
                qa = selected[0]
                for q in qa:
                    if len(set(q.keys()) | suppose) != len(suppose):
                        continue
                    if q['question'].lower() not in unique_questions:
                        if not ('answer' in q and isinstance(q['answer'], str)):
                            continue
                            
                        for k in suppose:
                            q[k] = q[k].encode("utf8", errors="surrogatepass").decode("utf8")

                        questions.append(q)
                        unique_questions.add(q['question'].lower())
            except:
                pass
            
        if not len(questions):
            continue
        
        data[i]['text'] = data[i]['text'].encode("utf8", errors="surrogatepass").decode("utf8")
        data[i]['questions'] = questions
        fopen_j.write(f'{json.dumps(data[i])}\n')

100%|███████████████████████████████████████████████████████████████████████████| 2608/2608 [00:00<00:00, 5209.52it/s]


In [9]:
!wc -l generate-lom.agc.gov.my-open-qa-llama3.jsonl

2253 generate-lom.agc.gov.my-open-qa-llama3.jsonl


In [10]:
!ls -lh generate-lom.agc.gov.my-open-qa-llama3.jsonl

-rw-rw-r-- 1 husein husein 23M Mei  30 14:23 generate-lom.agc.gov.my-open-qa-llama3.jsonl


In [11]:
from huggingface_hub import HfApi
api = HfApi()
api.upload_file(
    path_or_fileobj='generate-lom.agc.gov.my-open-qa-llama3.jsonl',
    path_in_repo='generate-lom.agc.gov.my-open-qa-llama3.jsonl',
    repo_id='mesolitica/llama-3-70b-qa',
    repo_type='dataset',
)

generate-lom.agc.gov.my-open-qa-llama3.jsonl:   0%|          | 0.00/23.1M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/mesolitica/llama-3-70b-qa/commit/c8918473ce3728a5169e874284b5bd14e43d7c10', commit_message='Upload generate-lom.agc.gov.my-open-qa-llama3.jsonl with huggingface_hub', commit_description='', oid='c8918473ce3728a5169e874284b5bd14e43d7c10', pr_url=None, pr_revision=None, pr_num=None)